In [1]:
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

#  set max display width too view full text
pd.set_option("display.max_colwidth", None)

In [2]:
df_dir = "../../data/coral/post_edit/"

In [3]:
def load_dataset(pre_filtered_df_dir, transformation):
    dataset_path = os.path.join(
        pre_filtered_df_dir,
        f"{transformation}.parquet",
    )

    if not os.path.exists(dataset_path):
        print(f"Dataset not found at path: {dataset_path}")
        return None

    return pd.read_parquet(dataset_path)


def merge_datasets_with_transformation(pre_filtered_df_dir):
    transformations = ["original", "brand_to_generic", "generic_to_brand"]
    merged_df = pd.DataFrame()

    for transformation in transformations:
        df = load_dataset(pre_filtered_df_dir, transformation)
        if df is not None:
            df["transformation_type"] = transformation
            df["question_index"] = range(len(df))
            merged_df = pd.concat([merged_df, df], ignore_index=True)

    return merged_df


merged_df = merge_datasets_with_transformation(df_dir)
merged_df = merged_df.sort_values(by=["coral_idx"])

if not merged_df.empty:
    print(f"\nMerged Dataset")
    print(f"Contains {len(merged_df)} rows.")
else:
    print(f"Failed to load and merge datasets for inspection.")


Merged Dataset
Contains 555 rows.


In [4]:
merged_df.head(1)

coral_idx     Sex                 UCSFDerivedRaceEthnicity_X   BirthDate  \
257          0  Female  Native Hawaiian or Other Pacific Islander  1964-03-25   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [5]:
import os
import pandas as pd
from collections import Counter

# Assuming df_dir is defined and dataset_path is constructed correctly
dataset_path = os.path.join(df_dir, "original.parquet")

# Read the original dataset
original_df = pd.read_parquet(dataset_path)

# Flatten the list of keywords and count occurrences
all_keywords = [
    keyword for sublist in original_df["found_keywords"] for keyword in sublist
]
keyword_counts = Counter(all_keywords)

# Optionally, display the counts in a more readable format
keyword_counts_df = pd.DataFrame(keyword_counts.items(), columns=["Keyword", "Count"])
keyword_counts_df = keyword_counts_df.sort_values(by="Count", ascending=False)
print(keyword_counts_df)

# print number unique keywords
print(f"Number of unique keywords: {len(keyword_counts)}")

# Print the total count of keywords
total_keywords_count = sum(keyword_counts.values())
print(f"Total count of keywords: {total_keywords_count}")

                   Keyword  Count
4              gemcitabine     82
13              paclitaxel     50
11                abraxane     37
31               tamoxifen     34
5             capecitabine     33
34               letrozole     31
10              omeprazole     29
12                prilosec     27
2               irinotecan     26
6              oxaliplatin     24
54                   taxol     23
21                  xeloda     15
36                    dexa     15
18           dexamethasone     15
32                  femara     14
64               denosumab     13
45                arimidex     11
40             trastuzumab     11
15                decadron     11
52                 cytoxan     10
16             carboplatin     10
9               prednisone     10
46             amoxicillin     10
27          hydrocortisone     10
72              everolimus      9
47              exemestane      9
53             anastrozole      8
51                taxotere      8
82            

In [6]:
# save original df to csv for inspection
original_df.to_csv("../../data/coral/post_edit/original.csv", index=False)

In [8]:
import pandas as pd
import re


def extract_assessment_plan(note_text):
    # List of phrases to search for, formatted to handle special characters and spaces
    phrases = [
        "ASSESSMENT & PLAN",
        "ASSESSMENT / PLAN",
        "ASSESSMENT \\ PLAN",
        "ASSESSMENT AND PLAN",
        "ASSESSMENT",
        "FINAL IMPRESSION",
        "IMPRESSION",
        "PLAN",
    ]

    # Normalize the note_text to lower case and remove special characters
    note_text_normalized = re.sub(r"\s+", " ", note_text.strip()).lower()

    # Combine the phrases into a single regex pattern, formatted to lower case and handle special characters
    pattern = (
        r"(" + "|".join(re.escape(phrase.lower()) for phrase in phrases) + r"):(.*)"
    )

    # Search for the pattern
    match = re.search(pattern, note_text_normalized, re.DOTALL)
    if match:
        extracted_text = match.group(2).strip()
        return extracted_text
    else:
        return None


# Assuming `original_df` is already defined as your original DataFrame
df = original_df.copy()

# Apply the function to the DataFrame and store the results
df["assessment_plan"] = df["note_text"].apply(extract_assessment_plan)

# Iterate through the DataFrame and print rows where no match is found
for index, row in df.iterrows():
    if row["assessment_plan"] is None:
        print(f"No match found for index {index}:")
        print(row["note_text"])
        print()  # For better readability between notes

# Optionally, display the DataFrame
# print(df[["note_text", "assessment_plan"]])

No match found for index 0:
 UCSF Cancer Center GI Medical Oncology Program  ***** ***** *****, ***** *****  ***** ***** ***** *****-*****  Phone: *****-*****-***** | Fax: *****-*****-*****     Follow-up *****  ***** : ***** *****, MD, *****      11/24/16    ***** had the pleasure of seeing ***** ***** ***** for follow up of metastatic pancreatic adenocarcinoma with peritoneal carcinomatosis at our GI Medical Oncology practice at the UCSF ***** ***** Family Comprehensive Cancer Center.    Patient Identification and Oncology History  ***** ***** ***** is a 61 y.o. female who presents to our GI Oncology practice at ***** ***** ***** Family Comprehensive Cancer Center for evaluation of metastatic pancreatic adenocarcinoma with peritoneal carcinomatosis.     Patient initially presented to ***** Community Hospital with loose BM, weight loss, abdominal pain and jaundice on 11/16/15. She was found to have elevated LFTs/hyperbilirubinemia (t.bili 8.7). An abdominal ultrasound was obtained whic

In [ ]:
# write out when happy
def write_df_to_parquet(output_dir, local_dataset_name, split, version, df):
    split_path = os.path.join(
        output_dir, local_dataset_name, split, f"{version}.parquet"
    )

    # Ensure the directory exists
    os.makedirs(os.path.dirname(split_path), exist_ok=True)

    # Write the DataFrame to Parquet
    df.to_parquet(split_path, index=False)
    print(f"DataFrame written to {split_path}")